# Mapping Data to a Cartesian Grid

In [ ]:
# In this notebook, we will show how to map data in Antenna coordinates to
# a Cartesian grid using Py-ART's map submodule.

In [ ]:
# First import needed modules
import matplotlib.pyplot as plt
import numpy as np
import pyart

In [ ]:
# Read the data that is in antenna coordinates.
radar = pyart.io.read("110635.mdv")

In [ ]:
# Py-ART has the Grid object which has characteristics that are similar to that of the
# Radar object, except that the data are stored in Cartesian coordinates instead of the
# radar's coordinates. Much like the Radar object, the fields are accessible by simply
# accessing grid.fields['field_name']['data'], and the object contains parameters such as the
# radar's location and projection.

In [ ]:
# We can create a Grid object from a Radar object by using pyart.map.grid_from_radars()
# Grid shape is the amount of points within a dimension. Grid limits is the dimension limits
# in meters.
grid = pyart.map.grid_from_radars(
    radar,
    grid_shape=(41, 201, 201),
    grid_limits=(
        (
            0.0,
            20000,
        ),
        (-100000.0, 100000.0),
        (-100000, 100000.0),
    ),
)

In [ ]:
# If with look at the coordinate data, we can see we have 201 points between
# -100000 and 100000 meters.
grid.x["data"]

In [ ]:
# In Py-ART's graphing suite, there is a display class similar to RadarMapDisplay,
# but for grids. To plot the grid:
fig = plt.figure(figsize=[8, 8])
display = pyart.graph.GridMapDisplay(grid)
display.plot_grid("reflectivity", level=3, vmin=-20, vmax=60)

In [ ]:
# We can even do both longitudnal and latitudnal slices. Change the lat parameter to view
# different portions of the storm!
display.plot_latitude_slice("reflectivity", lat=36.75, vmin=0, vmax=60)
plt.xlim([-50, 50])

In [ ]:
# As you can see, this Grid suffers from gaps in data as well as artificial "feathers" near the
# top that are simply the result of projecting the data to Cartesian coordinates. We would like
# to eliminate these as much as possible. Thankfully, Py-ART let's us, on top of specifying the grid
# resolution, also place in a custom radius of influence which controls how many points surrounding
# the gate are factored into the interpolation. We can also change the weighting function used to
#'nearest neighbor', 'barnes' or 'cressman'. Let's try increasing the minimum radius of influence to 1.75 km
# and use Cressman as the weighting function.
grid_better = pyart.map.grid_from_radars(
    radar,
    grid_shape=(41, 201, 201),
    grid_limits=(
        (
            0.0,
            20000,
        ),
        (-100000.0, 100000.0),
        (-100000, 100000.0),
    ),
    roi_func="dist_beam",
    min_radius=1750.0,
    weighting_function="cressman",
)

In [ ]:
# As you can see, most of the artifacts at higher altitudes have been smoothed out, but we
# have lost some spatial resolution as a result.
display = pyart.graph.GridMapDisplay(grid_better)
display.plot_latitude_slice("reflectivity", lat=36.75, vmin=0, vmax=60)
plt.xlim([-50, 50])